<a href="https://colab.research.google.com/github/kimjjoy2/NSMC/blob/main/NSMC(Naver_Sentiment_Movie_Corpus%2CNSMC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 네이버 영화리뷰 감정분석 데이터 다운로드
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 3.77 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [2]:
# transformers 설치
! pip install transformers

     |████████████████████████████████| 1.5MB 4.1MB/s 
     |████████████████████████████████| 890kB 19.9MB/s 
     |████████████████████████████████| 2.9MB 14.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=30fc7b2b2e09ae1ff79ed431d4b0739901461282702b46876cb0d8c406127766
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [39]:
# import
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [4]:
# GPU 셑팅
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
# tokenizer, model 설치
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

In [36]:
# 전처리
## NaN, 중복 제거
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file, sep, dup_field='document'):
  
    self.dataset = pd.read_csv(csv_file, sep=sep).dropna(axis=0) 
    if dup_field:
      self.dataset.drop_duplicates(subset=[dup_field], inplace=True)
    self.tokenizer = tokenizer

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:2].values
    text = row[0]
    if len(row) < 2:
      y = 0
    else:
      y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [37]:
# train, test dataset 준비
train_dataset = NSMCDataset("/content/nsmc/ratings_train.txt", '\t')
test_dataset = NSMCDataset("/content/nsmc/ratings_test.txt", '\t')
train_dataset[0]

                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  4.915700e+04  49157.000000
mean   6.752945e+06      0.502695
std    2.937158e+06      0.499998
min    6.010000e+02      0.000000
25%    4.777143e+06      0.000000
50%    7.565415e+06      1.000000
75%    9.260204e+06      1.000000
max    1.027809e+07      1.000000


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(tensor([    2,  3079, 33345,    18,    18,  7082, 13215,  4065,  4116,  4150,
          6933,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [10]:
# Train 환경 설정
epochs = 5
batch_size = 128
optimizer = AdamW(model.parameters(), lr=1e-5)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [11]:
#Training

losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 39.54772314429283 Accuracy: tensor(0.8272, device='cuda:0')
Batch Loss: 77.15680420398712 Accuracy: tensor(0.8327, device='cuda:0')
Batch Loss: 114.57897055149078 Accuracy: tensor(0.8346, device='cuda:0')
Batch Loss: 151.73331014811993 Accuracy: tensor(0.8355, device='cuda:0')
Batch Loss: 187.89915199577808 Accuracy: tensor(0.8361, device='cuda:0')
Batch Loss: 222.407447129488 Accuracy: tensor(0.8383, device='cuda:0')
Batch Loss: 257.82006898522377 Accuracy: tensor(0.8393, device='cuda:0')
Batch Loss: 292.12951800227165 Accuracy: tensor(0.8409, device='cuda:0')
Batch Loss: 326.4857897609472 Accuracy: tensor(0.8423, device='cuda:0')
Batch Loss: 361.23975248634815 Accuracy: tensor(0.8429, device='cuda:0')
Batch Loss: 394.9905428439379 Accuracy: tensor(0.8440, device='cuda:0')

Train Loss: 410.0738912373781 Accuracy: tensor(0.8445, device='cuda:0')


Batch Loss: 32.057014375925064 Accuracy: tensor(0.8645, device='cuda:0')
Batch Loss: 63.838134586811066 Accuracy: tensor(0.8650, device='cuda:0')
Batch Loss: 94.92490519583225 Accuracy: tensor(0.8656, device='cuda:0')
Batch Loss: 126.71246832609177 Accuracy: tensor(0.8661, device='cuda:0')
Batch Loss: 159.68792802095413 Accuracy: tensor(0.8647, device='cuda:0')
Batch Loss: 190.92941761016846 Accuracy: tensor(0.8649, device='cuda:0')
Batch Loss: 223.7032330185175 Accuracy: tensor(0.8640, device='cuda:0')
Batch Loss: 255.41407383978367 Accuracy: tensor(0.8639, device='cuda:0')
Batch Loss: 287.2298391163349 Accuracy: tensor(0.8636, device='cuda:0')
Batch Loss: 318.75011962652206 Accuracy: tensor(0.8636, device='cuda:0')
Batch Loss: 350.0363042652607 Accuracy: tensor(0.8639, device='cuda:0')

Train Loss: 363.39589600265026 Accuracy: tensor(0.8638, device='cuda:0')


Batch Loss: 29.946554616093636 Accuracy: tensor(0.8710, device='cuda:0')
Batch Loss: 59.74477434158325 Accuracy: tensor(0.8719, device='cuda:0')
Batch Loss: 90.0329241156578 Accuracy: tensor(0.8724, device='cuda:0')
Batch Loss: 119.34398737549782 Accuracy: tensor(0.8737, device='cuda:0')
Batch Loss: 148.88493762910366 Accuracy: tensor(0.8743, device='cuda:0')
Batch Loss: 178.36677925288677 Accuracy: tensor(0.8740, device='cuda:0')
Batch Loss: 207.64792439341545 Accuracy: tensor(0.8744, device='cuda:0')
Batch Loss: 237.4158999621868 Accuracy: tensor(0.8745, device='cuda:0')
Batch Loss: 267.37312826514244 Accuracy: tensor(0.8740, device='cuda:0')
Batch Loss: 296.7718577682972 Accuracy: tensor(0.8740, device='cuda:0')
Batch Loss: 325.96619476377964 Accuracy: tensor(0.8742, device='cuda:0')

Train Loss: 338.9267432540655 Accuracy: tensor(0.8745, device='cuda:0')


Batch Loss: 27.235774949193 Accuracy: tensor(0.8855, device='cuda:0')
Batch Loss: 55.141584292054176 Accuracy: tensor(0.8835, device='cuda:0')
Batch Loss: 82.67503805458546 Accuracy: tensor(0.8842, device='cuda:0')
Batch Loss: 110.35277557373047 Accuracy: tensor(0.8848, device='cuda:0')
Batch Loss: 138.73704355955124 Accuracy: tensor(0.8838, device='cuda:0')
Batch Loss: 166.74173234403133 Accuracy: tensor(0.8832, device='cuda:0')
Batch Loss: 195.2175326794386 Accuracy: tensor(0.8829, device='cuda:0')
Batch Loss: 223.2453471571207 Accuracy: tensor(0.8827, device='cuda:0')
Batch Loss: 251.5898679792881 Accuracy: tensor(0.8823, device='cuda:0')
Batch Loss: 279.6373321413994 Accuracy: tensor(0.8824, device='cuda:0')
Batch Loss: 307.59568867087364 Accuracy: tensor(0.8826, device='cuda:0')

Train Loss: 319.31719571352005 Accuracy: tensor(0.8828, device='cuda:0')


Batch Loss: 27.26050852239132 Accuracy: tensor(0.8845, device='cuda:0')
Batch Loss: 53.57659915089607 Accuracy: tensor(0.8875, device='cuda:0')
Batch Loss: 80.14930884540081 Accuracy: tensor(0.8887, device='cuda:0')
Batch Loss: 106.52159421145916 Accuracy: tensor(0.8892, device='cuda:0')
Batch Loss: 133.20816305279732 Accuracy: tensor(0.8890, device='cuda:0')
Batch Loss: 159.44643934071064 Accuracy: tensor(0.8890, device='cuda:0')
Batch Loss: 187.08316917717457 Accuracy: tensor(0.8879, device='cuda:0')
Batch Loss: 214.39069309830666 Accuracy: tensor(0.8877, device='cuda:0')
Batch Loss: 241.25693513453007 Accuracy: tensor(0.8877, device='cuda:0')
Batch Loss: 267.3111757710576 Accuracy: tensor(0.8878, device='cuda:0')
Batch Loss: 294.22119630128145 Accuracy: tensor(0.8877, device='cuda:0')

Train Loss: 305.79140647500753 Accuracy: tensor(0.8878, device='cuda:0')


In [12]:
# Test

model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



Accuracy: tensor(0.8724, device='cuda:0')


In [15]:
# 모델 저장

import os

model_dir = '/content/model/model_KoELECTRA/'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

print("Saving model to %s" % model_dir)

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)


Saving model to ./model/model_KoELECTRA/


('./model/model_KoELECTRA/tokenizer_config.json',
 './model/model_KoELECTRA/special_tokens_map.json',
 './model/model_KoELECTRA/vocab.txt',
 './model/model_KoELECTRA/added_tokens.json')

In [40]:
# kaggle competition의 test set에 대한 prediction 결과

output_dir = '/content/output/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)\

final_dataset = NSMCDataset("/content/ko_data.csv", ',', dup_field=None)

wp = open('/content/output/ko_data_output_20201223.csv', 'w')
wp.write('Id,Predicted\n')

seq = 0

for text, attention_mask, y in final_dataset:
  output = model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))
  logits = output[0]
  logit = logits.detach().cpu().numpy()

  wp.write('%d,%d\n' % (seq, np.argmax(logit)))
  seq += 1

wp.close() 

                 Id
count  11187.000000
mean    5593.000000
std     3229.553065
min        0.000000
25%     2796.500000
50%     5593.000000
75%     8389.500000
max    11186.000000
(tensor([    2,  6595,  6341,  3194,  4480,  4820,  6394, 10561,    18,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,    

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
